<a href="https://colab.research.google.com/github/tahmidawal/random/blob/main/MonteCarloDCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
!pip install yfinance
!pip install FundamentalAnalysis
import yfinance as yf
import fundamentalanalysis as fa 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [241]:
import pandas as pd
balanceSheet = pd.read_csv("/content/drive/MyDrive/model/bs.csv")
cashFlow = pd.read_csv("/content/drive/MyDrive/model/csf.csv")
incomeStatement = pd.read_csv("/content/drive/MyDrive/model/is.csv")
ticker = 'GOOG'
api = "384de232a09606f8ff4de3489690dc4c" #api key

cashFlow
balanceSheet = balanceSheet.iloc[:,::-1]
cashFlow = cashFlow.iloc[:,::-1]
incomeStatement = incomeStatement.iloc[:,::-1]

In [244]:
balanceSheet

,partition_0,netdebt,totaldebt,totalinvestments,totalliabilitiesandtotalequity,totalequity,minorityinterest,totalliabilitiesandstockholdersequity,totalstockholdersequity,othertotalstockholdersequity,...,shortterminvestments,cashandcashequivalents,period,calendaryear,accepteddate,fillingdate,reportedcurrency,symbol,date,id
0,balance-sheet-statement.snappy.parquet,4.653000e+09,2.663700e+10,1.246940e+11,3.582550e+11,2.536260e+11,0.0,3.582550e+11,2.536260e+11,6.625800e+10,...,9.427500e+10,2.198400e+10,Q3,2022,2022-10-25 21:32:59,2022-10-26,USD,GOOG,2022-09-30 00:00:00.000,GOOG2022Q3
1,balance-sheet-statement.snappy.parquet,1.079200e+10,2.872800e+10,1.377260e+11,3.551850e+11,2.554190e+11,0.0,3.551850e+11,2.554190e+11,0.000000e+00,...,1.070610e+11,1.793600e+10,Q2,2022,2022-07-26 19:29:36,2022-07-27,USD,GOOG,2022-06-30 00:00:00.000,GOOG2022Q2
2,balance-sheet-statement.snappy.parquet,7.535000e+09,2.842100e+10,1.436280e+11,3.570960e+11,2.540040e+11,0.0,3.570960e+11,2.540040e+11,0.000000e+00,...,1.130840e+11,2.088600e+10,Q1,2022,2022-04-26 20:51:20,2022-04-27,USD,GOOG,2022-03-31 00:00:00.000,GOOG2022Q1
3,balance-sheet-statement.snappy.parquet,7.450000e+09,2.839500e+10,1.482530e+11,3.592680e+11,2.516350e+11,0.0,3.592680e+11,2.516350e+11,0.000000e+00,...,1.187040e+11,2.094500e+10,Q4,2021,2022-02-01 21:08:02,2022-02-02,USD,GOOG,2021-12-31 00:00:00.000,GOOG2021Q4
4,balance-sheet-statement.snappy.parquet,4.141000e+09,2.786000e+10,1.443850e+11,3.474030e+11,2.445670e+11,0.0,3.474030e+11,2.445670e+11,0.000000e+00,...,1.182840e+11,2.371900e+10,Q3,2021,2021-10-26 19:47:06,2021-10-27,USD,GOOG,2021-09-30 00:00:00.000,GOOG2021Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,balance-sheet-statement.snappy.parquet,-7.531810e+08,2.910000e+05,0.000000e+00,4.497718e+09,3.953857e+09,0.0,4.497718e+09,3.953857e+09,2.837395e+09,...,2.194495e+09,7.534720e+08,Q2,2005,2005-06-30,2005-06-30,USD,GOOG,2005-06-30 00:00:00.000,GOOG2005Q2
70,balance-sheet-statement.snappy.parquet,-4.812620e+08,1.310000e+06,0.000000e+00,3.865199e+09,3.426862e+09,0.0,3.865199e+09,3.426862e+09,2.673131e+09,...,2.024726e+09,4.825720e+08,Q1,2005,2005-03-31,2005-03-31,USD,GOOG,2005-03-31 00:00:00.000,GOOG2005Q1
71,balance-sheet-statement.snappy.parquet,-4.249710e+08,1.902000e+06,0.000000e+00,3.313351e+09,2.929056e+09,0.0,3.313351e+09,2.929056e+09,2.582352e+09,...,1.705424e+09,4.268730e+08,Q4,2004,2004-12-31,2004-12-31,USD,GOOG,2004-12-31 00:00:00.000,GOOG2004Q4
72,balance-sheet-statement.snappy.parquet,-3.413800e+08,3.089000e+06,0.000000e+00,2.888518e+09,2.589023e+09,0.0,2.888518e+09,2.589023e+09,2.497299e+09,...,1.513887e+09,3.444690e+08,Q3,2004,2004-09-30,2004-09-30,USD,GOOG,2004-09-30 00:00:00.000,GOOG2004Q3


In [213]:

dcf_proj = pd.DataFrame({
    "revenue" : incomeStatement['revenue'],
    "netincome" : cashFlow["netincome"],
    "freecash" : cashFlow["freecashflow"]
}).transpose()

In [214]:
dcf_proj.loc["incomeMargin"] = dcf_proj.loc["revenue"]/dcf_proj.loc["netincome"]
dcf_proj.loc["FCFr"] = dcf_proj.loc["revenue"]/dcf_proj.loc["freecash"]
rev_change_mean = incomeStatement["revenue"].pct_change().mean()
rev_change_stdd = incomeStatement["revenue"].pct_change().std()
margin_change = dcf_proj.loc["incomeMargin", dcf_proj.loc["incomeMargin"] > 0].min()
fcf_change = dcf_proj.loc["FCFr", dcf_proj.loc["FCFr"] > 0].min()

tnx = yf.Ticker("^TNX") #use yfinance to get the 10yr treasury bond return
rfr = tnx.info["previousClose"] #use previousclose to estimate rfr
 

In [215]:
dcf_proj.loc['year'] = balanceSheet['calendaryear']
dcf_proj = dcf_proj.transpose()
#print(dcf_proj.columns.tolist())
dcf_proj = dcf_proj.dropna()
dcf_proj['year'] = dcf_proj['year'].astype('int')

In [216]:
dcf_proj = dcf_proj.set_index('year')

In [217]:
dcf_proj = dcf_proj.iloc[::2]
dcf_proj = dcf_proj.iloc[::2]
dcf_proj = dcf_proj.dropna()
dcf_proj = dcf_proj.replace(np.inf, 100)
dcf_proj 

,revenue,netincome,freecash,incomeMargin,FCFr
year,,,,,
2022,6.909200e+10,1.391000e+10,1.607700e+10,4.967074,4.297568
2021,6.511800e+10,1.893600e+10,1.872000e+10,3.438847,3.478526
2020,4.617300e+10,1.124700e+10,1.159700e+10,4.105361,3.981461
2019,4.049900e+10,7.068000e+09,8.734000e+09,5.729909,4.636936
2018,3.374000e+10,9.192000e+09,7.928000e+09,3.670583,4.255802
2017,2.777200e+10,6.732000e+09,6.334000e+09,4.125371,4.384591
2016,2.245100e+10,5.061000e+09,7.262000e+09,4.436080,3.091573
2015,1.867500e+10,3.979000e+09,3.634000e+09,4.693390,5.138965
2014,1.652300e+10,2.813000e+09,3.577000e+09,5.873800,4.619234


In [232]:
dcf_proj = dcf_proj.iloc[::-1]
dcf_proj = dcf_proj.transpose()
dcf_proj

year,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
FCFr,5.014511e+00,4.456209e+00,3.839056e+00,3.917907e+00,3.196732e+00,2.340821e+00,3.423872e+00,2.972477e+00,4.502235e+00,5.330351e+00,4.619234e+00,5.138965e+00,3.091573e+00,4.384591e+00,4.255802e+00,4.636936e+00,3.981461e+00,3.478526e+00,4.297568e+00
incomeMargin,1.550290e+01,4.140951e+00,3.667598e+00,3.954574e+00,4.295855e+00,3.627176e+00,1.000000e+02,1.000000e+02,6.480239e+00,5.014478e+00,5.873800e+00,4.693390e+00,4.436080e+00,4.125371e+00,3.670583e+00,5.729909e+00,4.105361e+00,3.438847e+00,4.967074e+00
freecash,1.607110e+08,3.542150e+08,7.006080e+08,1.080003e+09,1.733455e+09,2.539644e+09,2.128000e+09,3.270000e+09,3.132000e+09,2.794000e+09,3.577000e+09,3.634000e+09,7.262000e+09,6.334000e+09,7.928000e+09,8.734000e+09,1.159700e+10,1.872000e+10,1.607700e+10
netincome,5.198300e+07,3.811820e+08,7.333610e+08,1.069989e+09,1.289939e+09,1.638975e+09,0.000000e+00,0.000000e+00,2.176000e+09,2.970000e+09,2.813000e+09,3.979000e+09,5.061000e+09,6.732000e+09,9.192000e+09,7.068000e+09,1.124700e+10,1.893600e+10,1.391000e+10
revenue,8.058870e+08,1.578456e+09,2.689673e+09,4.231351e+09,5.541391e+09,5.944851e+09,7.286000e+09,9.720000e+09,1.410100e+10,1.489300e+10,1.652300e+10,1.867500e+10,2.245100e+10,2.777200e+10,3.374000e+10,4.049900e+10,4.617300e+10,6.511800e+10,6.909200e+10


In [237]:
year = 2022
#dcf_proj
startrev = dcf_proj.loc['revenue'][year] #get the starting revenue

allFVs = [] #create a list to store FV values

for p in range(10000): #run the simulation 10000 times
    #creating projections for a 5 year dcf model
    Fiveyrev = []
    Fiveyinc = []
    Fiveyfcf = []
    for i in range(5):
        startrev = startrev*(1+np.random.normal(loc=rev_change_mean, scale=rev_change_stdd))
        neti = startrev*(margin_change/100)
        fcf = neti*(fcf_change/100)
        Fiveyrev.append(startrev)
        Fiveyinc.append(neti)
        Fiveyfcf.append(fcf)



In [247]:
#get company profile for beta calculation and marketcap
profile = fa.profile(ticker, api)
quote = fa.quote(ticker, api)

beta = profile.loc["beta"] #get beta of stock from stock profile
mktCap = quote.loc["marketCap"] #market cap of stock
totdebt = balanceSheet["totaldebt",year] #total debt
sharesOut = quote.loc["sharesOutstanding"]
pg = 0.025 #perpetural growth

wacklist = []
for i,d in enumerate(["2018","2019","2020","2021","2022"]):
    Rd = ins.loc["interestExpense", d]/totdebt*(1-(ins.loc["incomeTaxExpense",d]/ins.loc["incomeBeforeTax",d]))
    Re = rfr + (beta*(10 - rfr))
    wacc = float(((Rd)*(totdebt/(mktCap + totdebt)) + (Re)*(1-(totdebt/(mktCap + totdebt))))/100)
    wacklist.append(Wacc)

wacc_stdd = np.std(wacklist)

KeyError: ignored